## The MNIST Dataset
**Author:** Rudra Nath Palit

## Reading Dataset
The data in the MNIST dataset is present in idx format. This is a binary format which makes it faster to access and less memory intensive compared to textual file systems such as `.csv`. The idx format is required to be loaded into numpy arrays for processing. The details regarding the format are given in the website itself.

In [2]:
import numpy as np
import struct as st

# Defining function to format idx to matrices
def openIdx(f):
    f.seek(0)
    ndims = st.unpack('>4B', f.read(4))[3]
    dims = np.zeros((ndims,), np.int64)
    for i in range(ndims):
        dims[i] = st.unpack('>I', f.read(4))[0]
    totalBytes = dims.prod()
    
    data = np.array(st.unpack('>' + 'B'*totalBytes, f.read(totalBytes))).reshape(dims)
    return data

# Opening files
img_train = open('data/train-images.idx3-ubyte', 'rb')
label_train = open('data/train-labels.idx1-ubyte', 'rb')
img_test = open('data/t10k-images.idx3-ubyte', 'rb')
label_test = open('data/t10k-labels.idx1-ubyte', 'rb')

x_train = openIdx(img_train)
y_train = openIdx(label_train)
x_test = openIdx(img_test)
y_test = openIdx(label_test)

# Print Data shape
print('x_train: {}\ty_train: {}\nx_test: {}\t\ty_test: {}'.format(x_train.shape, y_train.shape, x_test.shape, y_test.shape))

# Close Image handles
img_train.close()
label_train.close()
img_test.close()
label_test.close()

x_train: (60000, 28, 28)	y_train: (60000,)
x_test: (10000, 28, 28)		y_test: (10000,)


## Data Preparation
Here, we are normalizing the image data ranging from 0 to 255 and convert it to a range between 0-1. This helps to converge faster. Moreover, we are converting the 2D Image data of shape 28x28 into a 1D vector of length 784.

In [3]:
# Normalizing data
x_train_norm = x_train.astype(np.float32) /255.0 
x_test_norm = x_test.astype(np.float32) /255.0

# Seralize Data
x_train_ser = x_train_norm.reshape(x_train_norm.shape[0], -1)
x_test_ser = x_test_norm.reshape(x_test_norm.shape[0], -1)

## cNN Development
Here, we are developing a two hidden layer Neural Network with 128 Nodes each. The input contains 28x28 = 784 nodes and the output has 10 Nodes denoting the 10 digits in the number system. To implement this, we are using the Tensorflow framework with Keras. 

In [4]:
import tensorflow as tf

# The Net
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape = (784,), activation = 'relu'))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Training
model.fit(x = x_train_ser, y = y_train, epochs = 3)

# Save Model
model.save('mnist.model')

Epoch 1/3
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2299 - accuracy: 0.9316
Epoch 2/3
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0961 - accuracy: 0.9705
Epoch 3/3
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0665 - accuracy: 0.9787
INFO:tensorflow:Assets written to: mnist.model\assets


## Model Evaluation
Here, we are finally loading the trained model and evaluate its accuracy based on our test dataset.

In [6]:
model = tf.keras.models.load_model('mnist.model')
loss, accuracy = model.evaluate(x_test_ser, y_test)
print('Loss:\t\t{:.2f}\nAccuracy:\t{:.2f}%'.format(loss, accuracy*100))

313/313 [==============================] - 0s 930us/step - loss: 0.0787 - accuracy: 0.9771
Loss:		0.08
Accuracy:	97.71%
